# Agentic workflows using Llama-Index
The [Llama-Index](https://github.com/run-llama/llama_index) library and its submodules such as for [Ollama](https://docs.llamaindex.ai/en/stable/api_reference/llms/ollama/) and [OpenAI-like](https://docs.llamaindex.ai/en/stable/api_reference/llms/openai_like/) allow building agentic workflows using open-weight models. In such workflows the LLM decides which functions to call and it can call multiple functions in a row to answer a question.

In [1]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.ollama import Ollama
from llama_index.llms.openai_like import OpenAILike
from llama_index.core.tools import FunctionTool
import os

In the following example we use the OpenAI-like API to access our institutional LLM server. Alternatively, one can setup a local installation of [Ollama](https://ollama.com), which would work, too.

In [2]:
## Use this to play with a local model via ollama
#llm = Ollama(model="llama3.2", request_timeout=120.0)

llm = OpenAILike(model="meta-llama/Llama-3.3-70B-Instruct", request_timeout=120.0, api_base="https://llm.scads.ai/v1", api_key=os.environ.get('SCADSAI_API_KEY'))


We define a couple of tools, functions to answer specific questions. In the case shown here, we would like to deal with orders from customers and estimate their delivery dates when they approach us.

In [3]:
tools = []

@tools.append
def estimate_delivery_date_of_order(order_id:str) -> str:
    """Estimate the delivery date of a package identified by its order id."""
    if order_id == "292123":
        return "Friday"
    if order_id == "292456":
        return "Thursday"
    if order_id == "292789":
        return "Saturday"
    return "Unknown"

@tools.append
def get_recent_order_id(customer_name:str) -> str:
    """Get the most recent order id for a given customer"""
    if customer_name.lower() == "robert":
        return "292123"
    if customer_name.lower() == "alice":
        return "292456"
    if customer_name.lower() == "ivy":
        return "292789"
    if customer_name.lower() == "dennis":
        return "2921010"
    return "unknown"

We also add an unrelated function, which should never be called when dealing with customers and deliveries.

In [4]:
@tools.append
def get_weather(city:str)->str:
    """Returns the current weather in a given city."""
    return "sunny"

Next, we setup a [ReActAgent](https://docs.llamaindex.ai/en/stable/api_reference/agent/react/) to deal with the given function and prompts.

In [5]:
agent = ReActAgent.from_tools([FunctionTool.from_defaults(fn=t) for t in tools], llm=llm, verbose=True)

We can now ask questions to the agent. In verbose mode, you will see which functions it will call to answer our request.

In [6]:
response = agent.chat("My Name is Robert. I ordered something. And I would like to know when it will arrive.")
response.response

> Running step 89187412-52e5-43fb-8b04-1b17a14c4477. Step input: My Name is Robert. I ordered something. And I would like to know when it will arrive.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: get_recent_order_id
Action Input: {'customer_name': 'Robert'}
Observation: 292123
> Running step 490db98a-c140-46a6-a139-145656631264. Step input: None
Thought: I have the order id. Now I can use another tool to estimate the delivery date.
Action: estimate_delivery_date_of_order
Action Input: {'order_id': '292123'}
Observation: Friday
> Running step 20836298-e613-43e9-a86b-b3f4d80d2e51. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer
Answer: Your order will arrive on Friday.


'Your order will arrive on Friday.'

For demonstration purposes, we run this again.

In [7]:
agent = ReActAgent.from_tools([FunctionTool.from_defaults(fn=t) for t in tools], llm=llm)

response = agent.chat("My Name is Alice. I ordered something. And I would like to know when it will arrive.")
response.response

'Your order will arrive on Thursday.'